# Models with stages

In this example, we'll check out

- the structure of Feedbax staged models;
- how thir structure allows them to be modified with arbitrary interventions;
- how to write your own staged models.

All Feedbax models, staged or not, are defined as [Equinox]((/feedbax/examples/pytrees#equinox)) `Module` objects. These may be composed of other `Module` objects, forming a [nested structure](/feedbax/examples/1_train#examining-the-models-tree-structure) of model components and their parameters. 

Once a model object is constructed, we can call it like a function. We're allowed to do that because we [define its `__call__` method](https://docs.python.org/3/reference/datamodel.html#class-instances). 

## The structure of a staged model

A lot can happen in `__call__`. 

Consider the following simplified definition of [`feedbax.bodies.SimpleFeedback`][feedbax.bodies.SimpleFeedback], which is a model of a single time step in which a neural network, after receiving some sensory feedback, sends a command to a mechanical model:

```python
import equinox as eqx


class SimpleFeedbackState(eqx.Module):
    mechanics: MechanicsState
    net: NetworkState
    feedback: ChannelState


class SimpleFeedback(eqx.Module):
    """Model of one step around a feedback loop between a neural network 
    and a mechanical model.
    """
    mechanics: Mechanics 
    net: eqx.Module  
    feedback_channel: Channel
    where_feedback: Callable[[SimpleFeedbackState], PyTree] = (
        lambda state: state.mechanics.plant.skeleton
    )
    
    def __call__(
        self, 
        input: PyTree[Array],  
        state: SimpleFeedbackState, 
        key: PRNGKeyArray,
    ) -> SimpleFeedbackState:
                
        key1, key2 = jr.split(key)
        
        feedback_state = self.feedback_channel(
            self.where_feedback(state),
            state.feedback,
            key1,
        )
        
        network_state = self.net(
            (input, feedback_state.output), 
            state.network, 
            key2
        )
        
        mechanics_state = self.mechanics(
            network_state.output, 
            state.mechanics
        )        
        
        return SimpleFeedbackState(
            mechanics=mechanics_state, 
            network=network_state,
            feedback=feedback_state,
        )
    
    # ...
    # We've omitted a bunch of other stuff from this definition!
```

First, notice that `SimpleFeedback` is an Equinox `Module` subclass. It's not obvious from this code alone, but [`Mechanics`][feedbax.mechanics.Mechanics] and [`Channel`][feedbax.channel.Channel] are also `Module` subclasses, with their own parameters and submodules. 

Observe the following about `__call__`. It:

- takes a `SimpleFeedbackState` object, and returns a new one. This is where "the state of the model" is actually stored, not in `SimpleFeedback` itself.
- also takes an argument `input`. This argument is a PyTree that includes any inputs to the model that aren't part of its state.
- contains several steps. Each step involves calling one of the components of the model, such as `self.feedback_channel` and `self.net`. Each component is passed some part of the model state, and returns an updated version of that part. Like the parent model, each component also takes an argument of additional information as input. 

    !!! quote ""    
        
        The components of the model that are called during `__call__` are
    
        === "`self.feedback_channel`"
            
            - takes as input `self.where_feedback(state)`, which is the part of `state` we want to store in the state of the feedback channel, to be retrieved on some later time step, depending on the delay.
            - takes `state.feedback` as its prior state. This is a `ChannelState` object.
            - returns an updated `Channel_State`, which we assign to `feedback_state`.
            
            Note that the default for `self.where_feedback` is `lambda state: state.mechanics.plant.skeleton`, which means that our sensory feedback consists of the full state of the skeleton—typically, the positions and velocities of some joints.
            
        === "`self.net`"
        
            - takes as input `(input, feedback_state.output)`. Here, `input` is the entire argument passed to `__call__` itself. Since `SimpleFeedback` is a top-level model, its `input` will consist of the trial-by-trial information needed to complete the task. The neural network is proper recipient of this information, in this case.
            - takes `state.network` as prior state. This is a `NetworkState` object.
            - returns an updated `NetworkState`, which we assign to `network_state`.
            
            This is the only step in the model that receives the `input` that was passed to `SimpleFeedback` itself. This is because the input to the model is typically information the network needs to complete the task—say, the position of the goal it should reach to. The input to all of the other model steps is some other part of the model state.
        
        === "`self.mechanics`"

            - takes as input `network_state.output`, where `network_state` contains the updated `NetworkState` returned by `self.net`: `network_state.output` is the command we want to send to the mechanical model.
            - takes `state.mechanics` as its prior state. This is a `MechanicsState` object.
            - returns an updated `MechanicsState`, which we assign to `mechanics_state`.
        
- builds and returns a new `SimpleFeedbackState`, after calling all the model steps and getting their updates to various parts of the state.

### Trying to intervene

What if we want to interfere with the command the neural network generates, after we call `self.net` but before we call `self.mechanics`? We could write a new module with an extra component that operates on `NetworkState`, and call it at the right moment:

```py hl_lines="1 8 31 32"
class SimpleFeedbackPerturbNetworkOutput(eqx.Module):
    net: eqx.Module  
    mechanics: Mechanics 
    feedback_channel: Channel
    where_feedback: Callable[[SimpleFeedbackState], PyTree] = (
        lambda state: state.mechanics.plant.skeleton
    )
    intervention: eqx.Module
    
    def __call__(
        self, 
        input: PyTree[Array],  
        state: SimpleFeedbackState, 
        key: PRNGKeyArray,
    ) -> SimpleFeedbackState:
                
        key1, key2 = jr.split(key)
        
        feedback_state = self.feedback_channel(
            self.where_feedback(state),
            state.feedback,
            key1,
        )
        
        network_state = self.net(
            (input, feedback_state.output), 
            state.network, 
            key2
        )
        
        # modifies `network_state.output` somehow
        network_state = self.intervention(network_state)
        
        mechanics_state = self.mechanics(
            network_state.output, 
            state.mechanics
        )        
        
        return SimpleFeedbackState(
            mechanics=mechanics_state, 
            network=network_state,
            feedback=feedback_state,
        )
```

It would be pretty inconvenient to have to do this every time we want to intervene a little. Once we have a model, it'd be nice to experiment on it quickly. 

Also, if we have a different model that's similar enough to `SimpleFeedback` that it might make sense to use the same kind of `NetworkState` intervention on it that we just used, we wouldn't want to have to manually rewrite that model too. 

Thankfully we can do something about this. 

### A more general way to intervene

Start by noticing that each step in the `__call__` method of our original `SimpleFeedback`:

- is defined as a modification of some part of the model state: each operation we perform returns some part of `SimpleFeedbackState`;
- calls a model component in a consistent way: no matter if we're calling `self.feedback_channel`, `self.net`, or `self.mechanics`, our call always looks like `self.something(input_to_something, state_associated_with_something, key)`.

!!! Note inline end ""
    Each component will also need a `key`, but we won't need to specify how that works for each component individually. 
    
    We'll just have to be sure to split up the `key` passed to `__call__`, so that each component gets a different key.

That means we can define each step in `__call__` with three pieces of information: 

1. Which model component to call—say, `self.net`;
2. How to select the input to that model component, from the full `input` and `state` passed to `SimpleFeedback`;
3. How to select the state associated with (and modified by) that model component, out of the full `state` passed to `SimpleFeedback`.

OK, let's try to do that. We'll define an object called `ModelStage` which holds the three pieces of information required to define each model stage. Then we'll define a `model_spec` that defines all the stages of our model.

```python
class ModelStage(eqx.Module):
    component: Callable
    where_input: Callable
    where_state: Callable


model_spec = dict({
    'update_feedback': ModelStage(
        # See explanation below for why we define this as a lambda!
        func=lambda self: self.feedback_channel,  
        where_input=lambda input, state: self.where_feedback(state),
        where_state=lambda state: state.feedback,  
    ),
    'net_step': ModelStage(
        func=lambda self: self.net,
        where_input=lambda input, state: (input, state.feedback.output),
        where_state=lambda state: state.net,                
    ),
    'mechanics_step': ModelStage(
        func=lambda self: self.mechanics,
        where_input=lambda input, state: state.net.output,
        where_state=lambda state: state.mechanics,
    ),
})       
```

!!! NOTE   
    Each of the fields in `ModelStage` is typed as `Callable`, which means it can be a function, a method, or any object with `__call__` defined. In this case, we define them inline as `lambda` functions.
    
    For `where_input` and `where_state`, this is similar to what we've seen in [earlier examples](/feedbax/examples/1_train#selecting-part-of-the-model-to-train). For example, `where_input` will take the `input` and `state` passed to `__call__`, and return the parts to be passed to the current stage's `func`.
    
    Why do we define `func` as `#!py lambda self: self.something` rather than just `#!py self.something`? It's to make sure that references to the component "stay fresh". If that doesn't make sense to you, don't worry about it at this point. Just remember that if you write your own staged models, you will need to write your `model_spec` this way.

In order to insert interventions at arbitrary points, here's what we'll do: 

1. include `model_spec` as an attribute of `SimpleFeedback` itself;
2. define `__call__` so that it calls each of the components in `model_spec`, passing them their respective subsets of the input and state, and using their return value to update the model state.    

    !!! Warning ""    
    
        Importantly, the way we define `__call__` will no longer allow our model stages to assign, or refer, to intermediate variables like `feedback_state`. This is why in the `model_spec` we just defined, the input to `self.net` includes `state.feedback.output`, where in our original definition of `SimpleFeedback` we had passed `feedback_state.output`.
        
        In our new `__call__`, we'll update the full model state *immediately* after each stage, rather than assigning to intermediate variables and then finally constructing a new `SimpleFeedbackState`. Every stage's inputs are only selected out of the full model state, not out of intermediate copies of parts of the state.
        
3. give `SimpleFeedback` a new attribute `intervenors`, where we can insert additional components that intervene on the model's state, *given the name of the model stage they should be applied before*. For example, if this attribute is set to `{'mechanics_step': [some_intervention, some_other_intervention]}` then `some_intervention` and `some_other_intervention` would be called one after the other, *immediately before* `self.mechanics` is called.



<a id='simple_feedback_general_def'></a>
```python
class SimpleFeedback(eqx.Module):
    net: eqx.Module  
    mechanics: Mechanics 
    feedback_channel: Channel
    where_feedback: Callable[[SimpleFeedbackState], PyTree] = \
        lambda state: state.mechanics.plant.skeleton
    intervenors: dict[str, eqx.Module]    
    
    @property
    def model_spec(self):
        return dict({
            'update_feedback': ModelStage(
                func=lambda self: self.feedback_channel,  
                where_input=lambda input, state: self.where_feedback(state),
                where_state=lambda state: state.feedback,  
            ),
            'net_step': ModelStage(
                func=lambda self: self.net,
                where_input=lambda input, state: (input, state.feedback.output),
                where_state=lambda state: state.net,                
            ),
            'mechanics_step': ModelStage(
                func=lambda self: self.mechanics,
                where_input=lambda input, state: state.net.output,
                where_state=lambda state: state.mechanics,
            ),
        })    
    
    def __call__(
        self, 
        input: PyTree[Array],  
        state: SimpleFeedbackState, 
        key: PRNGKeyArray,
    ) -> SimpleFeedbackState: 
    
        # Get a different key for each stage of the model.
        keys = jr.split(key, len(self.model_spec))
        
        # Loop through the model stages, pairing them with their keys.
        for (label, stage), key_stage in zip(self.model_spec.items(), keys):
            
            # Loop through all intervenors assigned to this model stage.
            for intervenor in self.intervenors[label]:
                state = intervenor(state)
            
            # Get the updated part of the state associated with the stage
            new_component_state = stage.func(
                stage.where_input(input, state),
                stage.where_state(state),
                key_stage,
            )
            
            # Modify the full model state
            state = eqx.tree_at(
                stage.where_state,  # Part to modify
                state,  # What is modified (full state)
                new_component_state,  # New part to insert
            )
        
        return state
```

Our model is now structured so that it's possible to [insert interventions](/feedbax/examples/3_intervening/#adding-a-force-field) among its stages, without rewriting the whole thing each time!

This `__call__` method is too general to be limited to `SimpleFeedback`. In fact, the real [`feedbax.bodies.SimpleFeedback`][feedbax.bodies.SimpleFeedback] doesn't define `__call__` itself, but inherits it from [`feedbax.AbstractStagedModel`][feedbax.AbstractStagedModel]. 

Every staged model is a subclass of `AbstractStagedModel`, and only needs to define `model_spec` (and a couple of other smaller things).

Defining models as a sequence of named state operations has some additional advantages, beyond being able to insert interventions among the stages. For one, it makes it easy to [log the details](/feedbax/examples/debugging/#logging-details-of-model-execution) of our model stages as they are executed, which is useful for debugging.

## Pretty printing of model stages

Another advantage of staged models is that it's easy to print out a tree of operations, showing the sequence in which the're performed.

Feedbax provides the function [`pprint_model_spec`][feedbax.pprint_model_spec] for this purpose.

In [3]:
import jax
from feedbax import pprint_model_spec
from feedbax.xabdeef import point_mass_nn_simple_reaches

context = point_mass_nn_simple_reaches(key=jax.random.PRNGKey(0))

pprint_model_spec(context.model.step)

update_feedback: MultiModel
nn_step: SimpleStagedNetwork
  hidden: GRUCell
  readout: SimpleStagedNetwork._output
mechanics_step: Mechanics
  convert_effector_force: PointMass.update_state_given_effector_force
  statics_step: DirectForceInput
    clip_skeleton_state: DirectForceInput._clip_state
  dynamics_step: Mechanics._dynamics_step
  get_effector: PointMass.effector


Each line corresponds to a model stage. When the component for that stage is also a staged model, its own stages are printed on the lines that follow, with indentation. For example, `"clip_skeleton_state"` is a stage of `DirectForceInput`, which is called as part of the `"statics_step"` stage of `Mechanics`.

## Writing a staged model

The following components are needed to define your own staged model.

1. A [final](https://docs.kidger.site/equinox/pattern/) subclass of [`AbstractState`][feedbax.AbstractState] that defines the states that the model will be able to operate on. The fields of this subclass are typically JAX arrays, or other subclasses of `AbstractState` that are associated with the model's components. For example, if the model has a `Mechanics` component, its state should have a `MechanicsState` field.
2. A final subclass of [`AbstractStagedModel`][feedbax.AbstractStagedModel]. This subclass must implement 
    - a `model_spec` property defining, as above, the information needed to call the model stages;
    - an `init` method that takes a random key, and returns a default model state of the type defined in 1;
    - the field `intervenors: Mapping[str, Sequence[AbstractIntervenor]]`, where intervenors will be stored. 
    
For example, here's how to define a staged model that contains two neural networks in a loop, where the second network receives the first network's output after some delay, and at the beginning of the next iteration (time step) the

In [ ]:
from collections import OrderedDict
from collections.abc import Mapping, Sequence

import jax
from jaxtyping import PRNGKeyArray

from feedbax import AbstractStagedModel, ModelStage
from feedbax.channel import Channel, ChannelState
from feedbax.intervene import AbstractIntervenor
from feedbax.nn import SimpleStagedNetwork, NetworkState
from feedbax.state import AbstractState


class NetworkLoopState(AbstractState):
    net1: NetworkState
    net2: NetworkState
    channel: ChannelState


class NetworkLoop(AbstractStagedModel[NetworkLoopState]):
    net1: SimpleStagedNetwork
    net2: SimpleStagedNetwork
    channel: Channel
    intervenors: dict[str, Sequence[AbstractIntervenor]]

    @property
    def model_spec(self) -> OrderedDict[str, ModelStage]:
        return OrderedDict({
            'net1_step': ModelStage(
                callable=lambda self: self.net1,
                where_input=lambda input, state: state.net2.output,
                where_output=lambda state: state.net1,
            ),
            'channel': ModelStage(
                callable=lambda self: self.channel,
                where_input=lambda input, state: state.net1.output,
                where_output=lambda state: state.channel,
            ),
            'net2_step': ModelStage(
                callable=lambda self: self.net2,
                where_input=lambda input, state: state.channel.output,
                where_output=lambda state: state.net2,
            ),
        })

    def init(self, *, key: PRNGKeyArray | None = None) -> NetworkLoopState:
        keys = jax.random.split(key, 3)
        return NetworkLoopState(
            net1=self.net1.init(key=keys[0]),
            net2=self.net2.init(key=keys[1]),
            channel=self.channel.init(key=keys[2]),
        )

To construct this model, we have to construct its components. Normally we write a [setup function](/feedbax/examples/saving_and_loading) to make this reproducible.

In [ ]:
import jax.numpy as jnp
import jax.random as jr


def setup(
    net1_hidden_size,
    net2_hidden_size,
    channel_delay=5,
    channel_noise_std=0.05,
    *,
    key,
):
    key1, key2 = jr.split(key)
    net1 = SimpleStagedNetwork(
        input_size=net2_hidden_size,
        hidden_size=net1_hidden_size,
        key=key1,
    )
    net2 = SimpleStagedNetwork(
        input_size=net1_hidden_size,
        hidden_size=net2_hidden_size,
        key=key2
    )
    channel = Channel(
        channel_delay,
        channel_noise_std,
        input_proto=jnp.zeros(net2_hidden_size)
    )

    return NetworkLoop(net1=net1, net2=net2, channel=channel)

Of course, we haven't defined what we would be using such a model for. For one, notice that none of the stages of `NetworkLoop` pass any part of the `input` to their respective components—so the networks couldn't be directly receiving any task information. (However, we can still imagine optimizing for certain targets inside the network during training. And we could imagine that this is just a motif we would use as part of a larger model.)

### Using stateless modules and simple functions as components

Some components may be able to take one part of the model's state as input, and return another part as output, without having their own associated state. Often these are components whose inputs and outputs are single JAX arrays.

For example, consider the `"readout"` stage of `SimpleStagedNetwork`, which operates on `NetworkState`. The component for this stage is an `eqx.nn.Linear` layer. Notice that [`Linear.__call__`](https://docs.kidger.site/equinox/api/nn/linear/#equinox.nn.Linear.__call__) only takes a single argument, `x: Array`. This is the input array to the linear layer, which in our case corresponds to `state.hidden`—the activity of the hidden layer of our network. The output of the layer is used to update `state.output`.

Looking back at our generalized definition of `__call__` for staged models, each model stage is called like

!!! Note inline end ""
    As is generally the case for Feedbax and Equinox modules, `Linear` also has an argument `key`—but being a non-stochastic layer, will simply ignore it if passed.
    
    If the component doesn't take a `key` argument at all, the solution is similar to what we discuss below.
    
```python 
    # Get the updated part of the state associated with the stage
    new_component_state = stage.func(
        stage.where_input(input, state),
        stage.where_state(state),
        key_stage,
    )
```

Not including the key, *two* arguments are passed, after being selected by `where_input` and `where_state`. Clearly this will raise an error if `stage.func` refers to a single-argument module like `Linear`.

When we define our `model_spec`, we have to be careful in these cases. Here's a sketch of how we could define this stage:

```python
class StagedNetWithReadout(AbstractStagedModel[NetworkState]):
    readout: eqx.nn.Linear
    ...

    @property 
    def model_spec(self):
        return OrderedDict({
            ...,
            'readout': ModelStage(
                func=lambda self: (
                    lambda input, state, *, key: self.readout(input)
                ),
                where_input=lambda input, state: state.hidden,
                where_state=lambda state: state.output,
            )
            ...,
        })
```

We wrap `self.readout` inside a second lambda, which passes the `where_input(...)` argument on to `Linear`, but simply discards the `where_state(...)` argument. Note that `where_state` still needs to be defined, since it's also used to determine which part of the model's state to update with the *output* of the linear layer.

!!! Info ""
    If you find the double-lambda hard to read, you might prefer the function ['feedbax.wrap_stateless_callable'][feedbax.wrap_stateless_callable]:
    
    ```python 
    func=lambda self: wrap_stateless_callable(self.readout),
    ```    
    
    This function can also be used to discard a `key` that would be passed to the component:
    
    ```python 
    func=lambda self: wrap_stateless_callable(self.stateless_keyless_component, pass_key=False),
    ``` 


### Using non-staged components 

Say you have a complex neural network model which you'd like to use in Feedbax, but you don't want to rewrite the whole thing as a subclass of `AbstractStagedModel` with a `model_spec` that specifies operations on some type of `AbstractState`.

That's fine: you can still use such a network as a component. The downside is you won't be able to insert interventions inside the network as easily. 

(For example. Do we need a wrapper to make it work with `NetworkState`? Or should we make `SimpleFeedbackState` have a `NetworkState | Array` field?) 